In [18]:
import requests
import pandas as pd
#import json

Definerer funksjonen som henter data fra kart-API'et.

In [21]:
def getMapResponse(offset, **kwargs):
    '''Denne funksjonen henter inn limit antall GeoJSON objekter fra og med index offset, og returnerer et response-objekt.'''

    limit = kwargs.get("limit", 2000)
    inputalternativ = kwargs.get("inputalternativ", 1)

    match inputalternativ:
        case 1:
            #Denne inneholder 243 rader, men hver inneholder for lange objekter til å vises korrekt, så de blir kuttet av, og viser bare starten på strekninger.
            url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banelenke/MapServer/0/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
        case 2: 
            #Denne inneholder for mange rader, som hver er veldig korte, og blir for mange til at Powerapps kan vise dem korrekt. Den viser bare de første 2000.
            url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=Banenavn&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
        case 3:
            #Denne skal testes mer, inneholder en forenklet strekningsmodell
            url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/10/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
        case 4: 
            #Denne inneholder stasjoner, og ønskes brukt sammen med alt 3. Kan kanskje brukes sammen med 1?
            url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/2/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    payload = {}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)
    
    print("Lengde på API respons:", len(response.text))
    return response

Definerer og initialiserer parametere.

In [41]:
pd.options.display.max_colwidth = 50 #Default = 50
hentetAlt = False
limit = 2000
offset = 0
sporDF = pd.DataFrame(None)

#inputalternativ = 4

Main loop: 
Her looper vi gjennom tl vi har hentet alt kartdata.

In [42]:
for inputalternativ in [3,4]:
    print(inputalternativ)
    while not hentetAlt:
        SporRawJSON = getMapResponse(offset = offset, limit = limit, inputalternativ = inputalternativ).json()  
        #print("Innholdet i ""features"" ser slik ut:" , str(SporRawJSON["features"])[:50])
        tempdf = pd.json_normalize(SporRawJSON)
        sporDF = pd.concat([sporDF, tempdf.explode("features")], ignore_index=True)
        print(sporDF.shape)

        if "exceededTransferLimit" in SporRawJSON:
            if SporRawJSON["exceededTransferLimit"] == True:
                print(f"Transfer limit exceeded. Objekter nummer {offset}-{offset+limit} har blitt hentet. Kjører en runde til.")
        else:
            print("All data er hentet.")
            hentetAlt = True
        
        print("\n")
        offset += limit
    hentetAlt = False




3
Lengde på API respons: 141632
(44, 2)
All data er hentet.


4
Lengde på API respons: 42
(45, 2)
All data er hentet.




In [ ]:
#Sample response
'''
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [13.6299147114066, 66.1564617390492],
          [13.6298764104629, 66.1564525877301],
          [13.6299147114066, 66.1564617390492]
        ]
      },
      "properties": {
        "Banenavn": "Nordlandsbanen"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [5.73934403181721, 60.4593989800107],
          [5.73934270707218, 60.4593853650686],
          [5.73934512720757, 60.4593612516533],
          [5.73932950205534, 60.4592613030111],
          [5.73930664342178, 60.4591609077203],
          [5.73925735839471, 60.4590101723198],
          [5.73919006967426, 60.4588611528486],
          [5.73910527704731, 60.4587143108161],
          [5.73901556669164, 60.4585875023944]
        ]
      },
      "properties": {
        "Banenavn": "Bergensbanen"
      }
    }
  ]
  '''

In [43]:
sporDF.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'id': 4282, 'geometry': {'..."
1,FeatureCollection,"{'type': 'Feature', 'id': 4285, 'geometry': {'..."
2,FeatureCollection,"{'type': 'Feature', 'id': 4286, 'geometry': {'..."
3,FeatureCollection,"{'type': 'Feature', 'id': 4287, 'geometry': {'..."
4,FeatureCollection,"{'type': 'Feature', 'id': 4288, 'geometry': {'..."


In [47]:
def BaneEllerPunkt(row):
    if row["features"].apply(lambda x: x.get('geometry', None)).apply(lambda y: y.get('type', None)) == "LineString":
        return row.apply(lambda x: x.get('properties', None)).apply(lambda y: y.get('Banenavn', None))
    elif row["features"].apply(lambda x: x.get('geometry', None)).apply(lambda y: y.get('type', None)) == "Point":
        return row.apply(lambda x: x.get('properties', None)).apply(lambda y: y.get('Navn', None))

Henter ut banenavn fra properties, og oppretter en egen kolonne for det. Legger også til en kolonne hvor features blir konvertert til string, for å gjøre replace fra ' til ", og legger til en kolonne med len(strfeature).

In [48]:
#sporDF = sporDF.assign(label2 = sporDF.properties.get('Banenavn', None))
#Why doesn't above work?
#testcol = sporDF["features"].apply(lambda x: x.get('properties', None))
#print(testcol.apply(lambda y: y.get('Banenavn', None)))

#sporDF = sporDF.assign(label = sporDF["features"].apply(lambda x: x.get('properties', None)).apply(lambda y: y.get('Banenavn', None)))
sporDF = sporDF.assign(label = sporDF.apply(BaneEllerPunkt))

#sporDF = sporDF.assign(pkSpor=sporDF.index)

sporDF = sporDF.assign(strfeatures = sporDF['features'].astype("string").str.replace("'", '"') )
sporDF = sporDF.assign(strlen = sporDF["strfeatures"].map(len)) 

#Dropper alle andre kolonner enn de 3 vi er interesserte i.
sporDF = sporDF[["label", "strfeatures", "strlen"]]

#sporDF = sporDF.drop(columns=["features", "type"])
#if "label2" in sporDF.columns:
#    sporDF = sporDF.drop(columns="label2")
#if "exceededTransferLimit" in sporDF.columns:
#    sporDF = sporDF.drop(columns=["exceededTransferLimit", "properties.exceededTransferLimit"])

KeyError: 'features'

In [37]:
pd.options.display.max_colwidth = 30 #Default = 50

print(sporDF.shape)
print(sporDF.dtypes)
print(sporDF.sort_values("strlen", ascending=False))
#sporDF.head(10)

(44, 2)
type        object
features    object
dtype: object


KeyError: 'strlen'

In [27]:
sporDF.to_excel("GeoJSON-spor-forenklet-modell.xlsx")

Eksempel på json_normalize kode som illustrerer konseptet godt.

In [154]:
#Example json_normalize
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
print(pd.DataFrame(data))
result = pd.json_normalize(
    data, "counties", [["info", "governor"]])
print("result DF\n", result)

     state shortname                         info  \
0  Florida        FL   {'governor': 'Rick Scott'}   
1     Ohio        OH  {'governor': 'John Kasich'}   

                        counties  
0  [{'name': 'Dade', 'populat...  
1  [{'name': 'Summit', 'popul...  
result DF
          name  population info.governor
0        Dade       12345    Rick Scott
1     Broward       40000    Rick Scott
2  Palm Beach       60000    Rick Scott
3      Summit        1234   John Kasich
4    Cuyahoga        1337   John Kasich
